In [1]:
import os
import scipy.io as sio
import mat73
import numpy as np
from DecodingBasic import DataBasic
from functools import reduce
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
# subject info
sbj_name = 'YBN'
contrast_blk = ['037']
gamma_blk    = ['006']
recall_blk   = ['038', '039', '040']
tone_blk     = ['036']

# home directory
home_path = '~/Documents/MATLAB/ECoG'
home_dir  = os.path.expanduser(home_path)

In [3]:
# initiate decoding data object
train_contrast_obj = DataBasic(sbj_name, contrast_blk,'vis_contrast_only')
train_recall_obj   = DataBasic(sbj_name, recall_blk,'vis_contrast_recall')
train_gamma_obj    = DataBasic(sbj_name, gamma_blk,'vis_contrast_gamma')
test_recall_obj    = DataBasic(sbj_name, recall_blk,'vis_contrast_recall')
test_tone_obj      = DataBasic(sbj_name, tone_blk,'vis_tone_only')

In [4]:
# get electrode list
elec_vis_list  = train_contrast_obj.elec_record('vis')
elec_resp_list = train_contrast_obj.elec_record('resp') 

In [5]:
# frequency band that we are interested in
band_NBG = [20, 60]

In [19]:
# extract training data and testing data, apply normalization
count_ci = 0 
for ci in elec_resp_list:
    
    # ~~~training data~~~~
    
    # extract spectral data and compute band average value
    train_contrast_spect, label_train_contrast, _ = train_contrast_obj.extract_training_data(ci)    
    train_recall_spect,   label_train_recall, _   = train_recall_obj.extract_training_data(ci)
    train_gamma_spect,    label_train_gamma, _    = train_gamma_obj.extract_training_data(ci)

    train_contrast_NBG = train_contrast_obj.get_band_value(band_NBG, train_contrast_spect, ci)
    train_recall_NBG   = train_recall_obj.get_band_value(band_NBG, train_recall_spect, ci)
    train_gamma_NBG    = train_gamma_obj.get_band_value(band_NBG, train_gamma_spect, ci)
    
    training_data  = np.concatenate((train_contrast_NBG, train_recall_NBG, train_gamma_NBG), axis = 0)
    
    # normalization
    m = np.mean(training_data)
    z = np.std(training_data)
    training_data = (training_data - m)/z
    
    # ~~~~testing data~~~~
    
    # extract
    test_recall_spect,    label_test_recall, _    = test_recall_obj.extract_testing_data(ci)
    test_tone_spect,      label_test_tone, _      = test_tone_obj.extract_testing_data(ci)
    
    test_recall_NBG = test_recall_obj.get_band_value(band_NBG, test_recall_spect, ci)
    test_tone_NBG   = test_tone_obj.get_band_value(band_NBG, test_tone_spect, ci)
    
    testing_data  = np.concatenate((test_recall_NBG, test_tone_NBG), axis = 0)
    
    # normalization
    testing_data = (testing_data - m)/z
    
    # ~~~~append data~~~~~
    training_data = np.expand_dims(training_data, axis = 0)
    testing_data  = np.expand_dims(testing_data, axis = 0)
    
    if count_ci == 0:
        training_all = training_data
        testing_all  = testing_data
    else:
        training_all = np.append(training_all, training_data, axis = 0)
        testing_all  = np.append(testing_all,  testing_data,  axis = 0)
    
    # ~~~~~extract label~~~~~
    training_label = np.concatenate((label_train_contrast, label_train_recall, label_train_gamma))
    testing_label = np.concatenate((label_test_recall, label_test_tone))
    
    count_ci += 1
    

In [25]:
np.save('training_data_YBN', training_all)
np.save('testing_data_YBN', testing_all)
np.save('training_label_YBN', training_label)
np.save('testing_label_YBN', testing_label)

In [24]:
testing_label.shape

(120,)